In [5]:
from deeppavlov import build_model, configs
model = build_model(configs.syntax.ru_syntagrus_joint_parsing, download=False)

[nltk_data] Downloading package punkt to /home/kravtsova/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/kravtsova/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package perluniprops to
[nltk_data]     /home/kravtsova/nltk_data...
[nltk_data]   Package perluniprops is already up-to-date!
[nltk_data] Downloading package nonbreaking_prefixes to
[nltk_data]     /home/kravtsova/nltk_data...
[nltk_data]   Package nonbreaking_prefixes is already up-to-date!


2020-06-03 13:54:47.377 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 115: [loading vocabulary from /cephfs/home/kravtsova/.deeppavlov/models/morpho_ru_syntagrus/tag.dict]











The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
Please use `layer.__call__` method instead.

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where




Instructions for updating:
Use standard file APIs to check for files with this prefix.


2020-06-03 13:55:09.8 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 51: [loading model from /cephfs/home/kravtsova/.deeppavlov/models/morpho_ru_syntagrus/model]



INFO:tensorflow:Restoring parameters from /cephfs/home/kravtsova/.deeppavlov/models/morpho_ru_syntagrus/model


2020-06-03 13:55:13.183 WARNING in 'deeppavlov.core.models.serializable'['serializable'] at line 52: No load path is set for UDPymorphyLemmatizer!
2020-06-03 13:55:13.745 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 115: [loading vocabulary from /cephfs/home/kravtsova/.deeppavlov/models/syntax_ru_syntagrus/deps.dict]


Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.Bidirectional(keras.layers.RNN(cell))`, which is equivalent to this API
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Please use `layer.add_weight` method instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor



2020-06-03 13:55:35.212 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 51: [loading model from /cephfs/home/kravtsova/.deeppavlov/models/syntax_ru_syntagrus/model_joint]


INFO:tensorflow:Restoring parameters from /cephfs/home/kravtsova/.deeppavlov/models/syntax_ru_syntagrus/model_joint


In [6]:
sentences = ["Восемь дней спустя и националисты нападали на Каталонию ."]
for parse in model(sentences):
    print(parse, end="\n\n")

1	Восемь	восемь	NUM	_	Case=Acc	2	nummod:gov	_	_
2	дней	день	NOUN	_	Animacy=Inan|Case=Gen|Gender=Masc|Number=Plur	6	obl	_	_
3	спустя	спустя	ADP	_	_	2	case	_	_
4	и	и	PART	_	_	5	advmod	_	_
5	националисты	националист	NOUN	_	Animacy=Anim|Case=Nom|Gender=Masc|Number=Plur	6	nsubj	_	_
6	нападали	нападать	VERB	_	Aspect=Imp|Mood=Ind|Number=Plur|Tense=Past|VerbForm=Fin|Voice=Act	0	root	_	_
7	на	на	ADP	_	_	8	case	_	_
8	Каталонию	каталония	PROPN	_	Animacy=Inan|Case=Acc|Gender=Fem|Number=Sing	6	obl	_	_
9	.	.	PUNCT	_	_	6	punct	_	_



In [239]:
with open('RULEC-GEC_test_original_ya_corrected_parsed.txt', 'r', encoding='utf8') as f:
    test_ya_parsed = f.read()
test_parsed = test_ya_parsed.split('\n\n')
del test_parsed[-1]

In [8]:
with open('skills_2/RULEC-GEC_test_ya_speller_punct_comma-masked_prep-masked_parsed.txt', 'r', encoding='utf8') as f:
    test_ya_parsed = f.read()
test_parsed = test_ya_parsed.split('\n\n')
del test_parsed[-1]

In [7]:
with open('RULEC-GEC_test_original_ya_corrected.txt', 'r', encoding='utf8') as f:
    test = f.readlines()
test = [x.strip() for x in test]

#### Check noun wordforms (in case of incorrect morphotags) -- just test

In [9]:
for sent in tqdm(test_parsed):
    spl = sent.split('\n')
    synt_info = [x.split('\t') for x in spl]
    i = 0
    for i in range(len(synt_info)):
        tok = synt_info[i]
        if (tok[3] == 'NOUN') and tok[1][0].islower():
            first_letter = tok[1][0]
            noun_features = tok[5].split('|')
            try:
                lexem = morph.parse(tok[2])
                lex = [x for x in lexem if 'NOUN' in x.tag and 'nomn' in x.tag]
                if lex:
                    if 'Sgtm' in str(lex[0].tag):
    #                     i += 1
                        continue
                    fs = [features_dict[x.split('=')[1]] for x in noun_features if not x.startswith('Animacy') and not x.startswith('Gender')]
                    gender = [x for x in ['masc', 'femn', 'neut'] if x in str(lex[0].tag)]
                    fs.extend(gender)
                    if 'plur' in fs:
                        if 'masc' in fs:
                            fs.remove('masc')
                        elif 'femn' in fs:
                            fs.remove('femn')
                        elif 'neut' in fs:
                            fs.remove('neut')
                    test = first_letter + lex[0].inflect(set(fs)).word[1:]
                    synt_info[i][1] = test
    #                 print(test)
            except AttributeError:
                print('lazzzzzzhhhhaaa')
                print(tok)
                print(lex)
                print(fs)
                print(gender)
                print(sent)
#         i += 1
    new_sent = ' '.join(x[1] for x in synt_info)
#     print(new_sent)
    with open('skills_2/RULEC-GEC_test_ya_speller_punct_comma-masked_prep-masked_nouns-tags-check_test.txt', 'a', encoding='utf8') as f:
        print(new_sent, file=f, flush=True)

 94%|█████████▍| 4709/4993 [00:05<00:00, 938.57it/s] 

lazzzzzzhhhhaaa
['9', 'шахматы', 'шахматы', 'NOUN', '_', 'Animacy=Inan|Case=Dat|Gender=Fem|Number=Sing', '7', 'nmod', '_', '_']
[Parse(word='шахматы', tag=OpencorporaTag('NOUN,inan,GNdr,Pltm plur,nomn'), normal_form='шахматы', score=0.75, methods_stack=((<DictionaryAnalyzer>, 'шахматы', 209, 0),))]
['datv', 'sing']
[]
1	У	у	ADP	_	_	2	case	_	_
2	нас	мы	PRON	_	Case=Gen|Number=Plur|Person=1	6	obl	_	_
3	уже	уже	ADV	_	Degree=Pos	4	advmod	_	_
4	два	два	NUM	_	Animacy=Inan|Case=Acc|Gender=Masc	5	nummod:gov	_	_
5	года	год	NOUN	_	Animacy=Inan|Case=Gen|Gender=Masc|Number=Sing	6	obl	_	_
6	идёт	идти	VERB	_	Aspect=Imp|Mood=Ind|Number=Sing|Person=3|Tense=Pres|VerbForm=Fin|Voice=Act	0	root	_	_
7	соревнование	соревнование	NOUN	_	Animacy=Inan|Case=Nom|Gender=Neut|Number=Sing	6	nsubj	_	_
8	по	по	ADP	_	_	9	case	_	_
9	шахматы	шахматы	NOUN	_	Animacy=Inan|Case=Dat|Gender=Fem|Number=Sing	7	nmod	_	_
10	.	.	PUNCT	_	_	6	punct	_	_


100%|██████████| 4993/4993 [00:05<00:00, 899.56it/s] 


### Noun agreement with prep on syntactic tree using prep case -- skills_2

In [13]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

In [14]:
features_dict = {'Nom':'nomn', 'Gen':'gent', 'Dat':'datv', 'Acc':'accs', 'Ins':'ablt', 'Loc':'loct', 'Voc':'voct',
       'Par':'gen2', 'Sing':'sing', 'Plur':'plur', 'Masc':'masc', 'Fem':'femn', 'Neut':'neut', 'Sup':'Supr',
                 'Anim':'anim', 'Inan':'inan',
                'Short':'ADJS'}

In [15]:
lex = morph.parse('досада')[0]
lex
# 'ADJF' in lex.tag
# lex.inflect({'femn', 'sing', 'ablt'}).word

Parse(word='досада', tag=OpencorporaTag('NOUN,inan,femn sing,nomn'), normal_form='досада', score=1.0, methods_stack=((<DictionaryAnalyzer>, 'досада', 55, 0),))

In [16]:
import pandas as pd
preps_data = pd.read_csv('rus_preps.csv')
preps = preps_data['word'].tolist()
cases = preps_data['case'].tolist()

In [17]:
cases_match = {'acc':'Case=Acc', 'dat':'Case=Dat', 'gen':'Case=Gen', 'inst':'Case=Ins', 'prep':'Case=Loc'}

In [18]:
new_cases = [cases_match.get(item, item) for item in cases]

In [19]:
preps_cases = dict(zip(preps, new_cases))

In [20]:
preps_cases['согласно'] = 'Case=Dat'

In [21]:
# preps_cases['спустя'] = 'Case=Acc,Case=Gen'
preps_cases['спустя'] = 'Case=Acc'

In [22]:
preps_cases['в'] = 'Case=Acc,Case=Loc' # не нужен генитив по идее, генитив для подстройки
preps_cases['с'] = 'Case=Ins,Case=Gen'

In [23]:
preps_cases['на'] = 'Case=Loc,Case=Acc'

In [24]:
preps_cases['за'] = 'Case=Ins,Case=Acc'

In [25]:
preps_cases['через'] = 'Case=Acc,Case=Gen'
# preps_cases['через'] = 'Case=Acc'

In [26]:
preps_cases['со'] = 'Case=Ins,Case=Gen'

In [27]:
preps_cases['для'] = 'Case=Gen,Case=Acc'

In [28]:
preps_cases['под'] = 'Case=Ins,Case=Acc'

In [29]:
# Составные предлоги
preps_cases['в из'] = ''
preps_cases['на в'] = ''
preps_cases['спустя после'] = ''

preps_cases['без ведома'] = 'Case=Gen'
preps_cases['близко от'] = 'Case=Gen'
preps_cases['в виде'] = 'Case=Gen'
preps_cases['в дополнение'] = ''
preps_cases['в дополнение к'] = 'Case=Dat'
preps_cases['в зависимости от'] = 'Case=Gen'
preps_cases['в интересах'] = 'Case=Gen'
preps_cases['в качестве'] = 'Case=Gen'
preps_cases['в лице'] = 'Case=Gen'
preps_cases['в отличие от'] = 'Case=Gen'
preps_cases['в пользу'] = 'Case=Gen'
preps_cases['в отношении'] = 'Case=Gen'
preps_cases['в преддверии'] = 'Case=Gen'
preps_cases['в продолжение'] = 'Case=Gen'
preps_cases['в результате'] = 'Case=Gen'
preps_cases['в роли'] = 'Case=Gen'
preps_cases['в связи с'] = 'Case=Ins'
preps_cases['в силу'] = 'Case=Gen'
preps_cases['в случае'] = 'Case=Gen'
preps_cases['в соответствии с'] = 'Case=Ins'
preps_cases['в соответствии со'] = 'Case=Ins'
preps_cases['в течение'] = 'Case=Gen'
preps_cases['в целях'] = 'Case=Gen'
preps_cases['в частности'] = ''
preps_cases['в честь'] = 'Case=Gen'
preps_cases['вдоль по'] = 'Case=Dat'
preps_cases['в течение'] = 'Case=Gen'
preps_cases['в ходе'] = 'Case=Gen'
preps_cases['во'] = 'Case=Loc,Case=Acc'
preps_cases['во имя'] = 'Case=Gen'
preps_cases['во славу'] = 'Case=Gen'
preps_cases['вплоть до'] = 'Case=Gen'
preps_cases['впредь до'] = 'Case=Gen'
preps_cases['вслед за'] = 'Case=Ins'
preps_cases['впредь до'] = 'Case=Gen'
preps_cases['за вычетом'] = 'Case=Gen'
preps_cases['за исключением'] = 'Case=Gen'
preps_cases['за счёт'] = 'Case=Gen'
preps_cases['за счет'] = 'Case=Gen'
preps_cases['за течение'] = 'Case=Gen'
preps_cases['исходя из'] = 'Case=Gen'
# preps_cases['ко'] = 'Case=Dat'
preps_cases['ко'] = ''
preps_cases['лицом к лицу с'] = 'Case=Ins'
preps_cases['на благо'] = 'Case=Gen'
preps_cases['на виду у'] = 'Case=Gen'
preps_cases['на глазах у'] = 'Case=Gen'
preps_cases['на предмет'] = 'Case=Gen'
preps_cases['наряду с'] = 'Case=Ins'
preps_cases['насчет'] = 'Case=Gen'
preps_cases['начиная с'] = 'Case=Gen'
preps_cases['не без'] = 'Case=Gen'
preps_cases['не считая'] = 'Case=Gen'
preps_cases['невзирая на'] = 'Case=Асс'
preps_cases['недалеко от'] = 'Case=Gen'
preps_cases['независимо от'] = 'Case=Gen'
preps_cases['несмотря на'] = 'Case=Асс'
preps_cases['об'] = 'Case=Loc'
preps_cases['обо'] = 'Case=Loc,Case=Acc'
preps_cases['от имени'] = 'Case=Gen'
preps_cases['от лица'] = 'Case=Gen'
preps_cases['по линии'] = 'Case=Gen'
preps_cases['по мере'] = 'Case=Gen'
preps_cases['по направлению к'] = 'Case=Dat'
preps_cases['по отношению к'] = 'Case=Dat'
preps_cases['по поводу'] = 'Case=Gen'
preps_cases['по причине'] = 'Case=Gen'
preps_cases['по случаю'] = 'Case=Gen'
preps_cases['по сравнению с'] = 'Case=Ins'
preps_cases['по сравнению со'] = 'Case=Ins'
preps_cases['поблизости от'] = 'Case=Gen'
preps_cases['под видом'] = 'Case=Gen'
preps_cases['под эгидой'] = 'Case=Gen'
preps_cases['при помощи'] = 'Case=Gen'
preps_cases['применительно к'] = 'Case=Dat'
preps_cases['рядом с'] = 'Case=Ins'
preps_cases['с ведома'] = 'Case=Gen'
preps_cases['с помощью'] = 'Case=Gen'
preps_cases['с прицелом на'] = 'Case=Acc'
preps_cases['с точки зрения'] = 'Case=Gen'
preps_cases['с целью'] = 'Case=Gen'
preps_cases['следом за'] = 'Case=Ins'
preps_cases['смотря по'] = 'Case=Dat'
preps_cases['согласно с'] = 'Case=Ins'
preps_cases['со стороны'] = 'Case=Gen'
preps_cases['судя по'] = 'Case=Dat'

In [30]:
ks = sorted(list(preps_cases.keys()), key=len)

In [31]:
with open('russian_preps.txt', 'w', encoding='utf8') as f:
    for x in ks:
        f.write(x + '\n')

In [32]:
from collections import defaultdict

In [65]:
with open('skills_2/RULEC-GEC_test_ya_speller_punct_comma-masked-no-lowercase_prep-masked_prep-noun-masked_parsed.txt', 'r', encoding='utf8') as f:
    test_ya_parsed = f.read()
test_parsed = test_ya_parsed.split('\n\n')
del test_parsed[-1]

In [33]:
with open('skills_2/excluded_test_ya_speller_punct_prep-masked_prep-noun-masked_parsed.txt', 'r', encoding='utf8') as f:
    test_ya_parsed = f.read()
test_parsed = test_ya_parsed.split('\n\n')
del test_parsed[-1]

In [35]:
# PrepP agreement errors including составные предлоги и без числительных
joint_sents = []
err = 0
for sent in test_parsed:
    spl = sent.split('\n')
    synt_info = [x.split('\t') for x in spl]
    pos = [synt_info[i][3] for i in range(len(synt_info))]
    heads = [synt_info[i][6] for i in range(len(synt_info))]
    heads_pos = defaultdict(set)
    for c, i in zip(heads, pos):
        heads_pos[c].add(i)
    i = 0
    while i < len(synt_info):
        tok = synt_info[i]
        if (tok[3] == 'ADP') and (tok[7] == 'case') and (synt_info[int(tok[6])-1][3] == 'NOUN') and (
        synt_info[int(tok[6])-1][7] == 'obl'):
            first_letter = synt_info[int(tok[6])-1][1][0]
            start = i
            i += 1
            while i < len(synt_info) and synt_info[i][7] == 'fixed':
                i += 1
            prep = ' '.join([x[1] for x in synt_info[start:i]]).lower()
            if not preps_cases[prep]:
                i += 1
                continue
            prep_features = preps_cases[prep].split(',')
            noun_features = synt_info[int(tok[6])-1][5].split('|')
            diff = [x for x in noun_features if x in prep_features]
            try:
                if len(diff) != 1:
                    if len(prep_features) == 2:
                        i += 1
                        continue
                    if 'NUM' in heads_pos[tok[6]]:
                        i += 1
                        continue
                    err += 1
                    fs = [features_dict[x.split('=')[1]] for x in noun_features if not x.startswith('Case')]
                    fs.append(features_dict[prep_features[0].split('=')[1]])
                    print(prep)
                    print(fs)
                    lexem = morph.parse(synt_info[int(tok[6])-1][2])
                    lex = [x for x in lexem if 'NOUN' in x.tag and 'nomn' in x.tag][0]
                    test = first_letter + lex.inflect(set(fs)).word[1:]
                    synt_info[int(tok[6])-1][1] = test 
#                     print(sent)
                    print(test) 
            except AttributeError:
                print('lazzzzzzhhhhaaa')
                print(prep_features)
                print(noun_features)
                print(sent)
        i += 1
    joint_sents.append(' '.join(x[1] for x in synt_info))

In [36]:
err

0

In [68]:
with open('skills_2/RULEC-GEC_test_ya_speller_punct_comma-masked-no-lowercase_prep-masked_prep-noun-masked_prep-noun-agr.txt', 'w', encoding='utf8') as f:
    for sent in joint_sents:
        f.write(sent + '\n')

In [2]:
with open('RULEC-GEC_test_original_ya_corrected_prep_agreement.txt', 'r', encoding='utf8') as f:
    test_prep = f.readlines()
test_prep = [x.strip() for x in test_prep]

In [3]:
with open('RULEC-GEC_test_original_ya_corrected_prep_agreement_parsed.txt', 'w', encoding='utf8') as f:
    for sent in test_prep:
        f.write(model([sent])[0] + '\n\n')

### AdjNoun agreement errors (skills_2)

In [8]:
from collections import defaultdict

In [19]:
with open('skills_2/RULEC-GEC_test_ya_speller_punct_comma-masked-no-lowercase_prep-masked_prep-noun-masked_prep-noun-agr_frame-v+noun_adj-masked_parsed.txt', 'r', encoding='utf8') as f:
    test_ya_parsed = f.read()
test_parsed = test_ya_parsed.split('\n\n')
del test_parsed[-1]

In [20]:
len(test_parsed)

4993

In [21]:
#AdjNoun agreement errors
err = 0
adj_joint_sents = []
for sent in test_parsed:
    spl = sent.split('\n')
    synt_info = [x.split('\t') for x in spl]
    deps = [synt_info[i][0] for i in range(len(synt_info))]
    heads = [synt_info[i][6] for i in range(len(synt_info))]
    heads_deps = defaultdict(set)
    for h, d in zip(heads, deps):
        heads_deps[h].add(d)
    hd = dict(heads_deps)
#     print(hd)
    for x in synt_info:
        first_letter = x[1][0]
        if any(char.isdigit() for char in x[1]):
            continue
        if (x[3] == 'ADJ') and ((x[7] == 'amod') or (x[7] == 'root')) and (
            len(x[5].split('|')) > 3) and ((synt_info[int(x[6])-1][3] == 'NOUN') or (x[6] == '0')) and (
        (int(x[0]) - int(synt_info[int(x[6])-1][0])) != 1) and (synt_info[int(x[0])-2][3] != 'NUM'): 
#         synt_info[int(synt_info[int(x[6])-1][6])-1][3] != 'NOUN'):
#             print(x)
            try:                    
                try:
                    adj_deps = hd[str(x[0])]
                    if any((synt_info[int(ind)-1][3] == 'PRON') or (synt_info[int(ind)-1][3] == 'ADJ') or (
                        synt_info[int(ind)-1][3] == 'AUX') for ind in adj_deps):
                        continue
                except KeyError:
                    pass
                adj_features = x[5].split('|')
                if x[6] != '0':
#                     if synt_info[int(x[6])-1][7] == 'appos':
#                         new_case = [x for x in synt_info[int(synt_info[int(x[6])-1][6])-1][5].split('|') if x.startswith('Case')][0]
#                         tmp_noun_features = synt_info[int(x[6])-1][5].split('|')
#                         noun_features = [new_case if x.startswith('Case') else x for x in tmp_noun_features]
#                     else:
                    noun_features = synt_info[int(x[6])-1][5].split('|')
                else:
                    try:
                        nsubj_ind = [synt_info[int(ind)-1][0] for ind in adj_deps if synt_info[int(ind)-1][7] == 'nsubj'][0]
                        noun_features = synt_info[int(nsubj_ind)-1][5].split('|')
                    except IndexError:
                        continue
#                 print(common)
                common = list(set(noun_features).intersection(adj_features))
                diff = list(set(noun_features) - set(adj_features))
                adj_diff = list(set(adj_features) - set(noun_features))
                degree = [x for x in adj_diff if x.startswith('Degree=Sup') or x.startswith('Variant=Short')]
                full = common + diff + degree
#                 print(full)
#                 print(diff)
                if 'Variant=Short' in full:
                    full = [x for x in full if not x.startswith('Case')]
                if len(diff) > 1:
#                 if diff:
                    fs = [features_dict[x.split('=')[1]] for x in full if not x.startswith('Animacy')]
#                     print(fs)
                    if 'plur' in fs:
                        if 'masc' in fs:
                            fs.remove('masc')
                        elif 'femn' in fs:
                            fs.remove('femn')
                        elif 'neut' in fs:
                            fs.remove('neut')
                    lexem = morph.parse(x[2])
                    try:
#                         if 'accs' in fs and 'masc' in fs and 'sing' in fs:
#                             lex = [x for x in lexem if 'ADJF' in x.tag and 'accs' in x.tag][0]
#                         else:
                        lex = [x for x in lexem if 'ADJF' in x.tag][0]
                    except IndexError:
                        continue
                    err += 1
                    test = first_letter + lex.inflect(set(fs)).word[1:]
#                     print(test)
                    x[1] = test
#                     print(sent)
            except AttributeError:
                print('LAZZZHHHAAAA')
                print(degree)
                print(full)
                print(fs)
                print(lex)
                print(sent)
    adj_joint_sents.append(' '.join(x[1] for x in synt_info))

In [22]:
err

83

In [23]:
with open('skills_2/RULEC-GEC_test_ya_speller_punct_comma-masked-no-lowercase_prep-masked_prep-noun-masked_prep-noun-agr_frame-v+noun_adj-masked_adj-noun-agr.txt',
          'w', encoding='utf8') as f:
    for sent in adj_joint_sents:
        f.write(sent + '\n')

In [77]:
with open('RULEC-GEC_test_original_ya_corrected_with_adj_agreement.txt', 'w', encoding='utf8') as f:
    for sent in adj_joint_sents:
        f.write(sent + '\n')

In [12]:
with open('RULEC-GEC_test_original_ya_corrected_prep_adj_agreement.txt', 'w', encoding='utf8') as f:
    for sent in adj_joint_sents:
        f.write(sent + '\n')

In [16]:
with open('skills_2/RULEC-GEC_test_ya_speller_punct_comma-masked-no-lowercase_prep-masked_prep-noun-masked_prep-noun-agr_frame-v+noun_adj-masked.txt', 'r', encoding='utf8') as f:
    ya_prep_mask = f.readlines()
ya_prep_mask = [x.strip() for x in ya_prep_mask]

In [ ]:
with open('skills_2/RULEC-GEC_test_ya_speller_punct_comma-masked-no-lowercase_prep-masked_prep-noun-masked_prep-noun-agr_frame-v+noun_adj-masked_parsed.txt', 'w', encoding='utf8') as f:
    for sent in ya_prep_mask:
        f.write(model([sent])[0] + '\n\n')

In [37]:
with open('skills_2/excluded_test_ya_speller_punct_prep-masked_prep-noun-masked_frame-v+noun.txt', 'r', encoding='utf8') as f:
    excluded_test = f.readlines()
excluded_test = [x.strip() for x in excluded_test]

In [7]:
excluded_test = ['Хотя природные явления ( извержения , цветения водорослей , землетрясения , и т .',
'Экологи показывали страшные фотографии и дали неопределенные угрозы , которые могут происходить на АЭС до или после строительства .',
'плакат А . Николаева , за который арестовали его в 2012',
'Работал с 1966 до 2008',
'Общая площадь : 68 кв .',
'В связи с тем , что эра Сталина произошло вместе со становлении Магадана , Магадан также стал частью механизмы террора и тирании сталинской власти .',
'Какие плюсы и минусы существуют ? ( Независимость , гордость своей страной , возвращение национальной культуры / нереальность обучения на этих языках , падание от большого русско говоряшего мира , современность русского языка , и . т . д . )']


In [38]:
with open('skills_2/excluded_test_ya_speller_punct_prep-masked_prep-noun-masked_frame-v+noun_parsed.txt', 'w', encoding='utf8') as f:
    for sent in excluded_test:
        f.write(model([sent])[0] + '\n\n')
#         f.write(sent + '\n')

In [30]:
!pwd

/cephfs/home/kravtsova/hse/thesis/RozovskayaRothTACL2018-dataset
